In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QSlider, QHBoxLayout, QVBoxLayout, QPushButton, QFileDialog, QGridLayout
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
import cv2
import numpy as np

class DetectCircles(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        
    def initUI(self):
        # Create labels to display the images
        self.image_label = QLabel(self)
        self.edge_label = QLabel(self)
        
        # Create a label for the title
        self.title_label = QLabel('Finding circles in images using Hough Transform')
        self.title_label.setAlignment(Qt.AlignCenter)  # Center the title label
        
        # Create sliders to control the parameters of Hough transform
        self.param1_slider = QSlider(Qt.Horizontal)
        self.param1_slider.setMinimum(1)
        self.param1_slider.setMaximum(255)
        self.param1_slider.setValue(50)
        self.param2_slider = QSlider(Qt.Horizontal)
        self.param2_slider.setMinimum(1)
        self.param2_slider.setMaximum(50)
        self.param2_slider.setValue(30)
        
        # Create a button to load the image
        self.load_button = QPushButton('Load Image')
        self.load_button.clicked.connect(self.load_image)
        
        # Create a button to detect edges
        self.detect_button = QPushButton('Detect Edges')
        self.detect_button.clicked.connect(self.detect)
        
        # Create a layout for the sliders and buttons
        hbox = QHBoxLayout()
        hbox.addWidget(QLabel('Param1'))
        hbox.addWidget(self.param1_slider)
        hbox.addWidget(QLabel('Param2'))
        hbox.addWidget(self.param2_slider)
        
        # Create a grid layout for the images
        grid = QGridLayout()
        grid.addWidget(self.image_label, 0, 0)
        grid.addWidget(self.edge_label, 0, 1)
        
        vbox = QVBoxLayout()
        vbox.addWidget(self.title_label)  # Add the title label
        vbox.addLayout(hbox)
        vbox.addWidget(self.load_button)
        vbox.addWidget(self.detect_button)
        vbox.addLayout(grid)
        
        # Set the layout of the widget
        self.setLayout(vbox)
        # Set the window properties
        self.setGeometry(300, 300, 600, 600)
        self.setWindowTitle('Circular Hough Transform')
        self.show()
    def load_image(self):
        # Open a file dialog to select the image
        options = QFileDialog.Options()
        options |= QFileDialog.ReadOnly
        file_name, _ = QFileDialog.getOpenFileName(self,"Select an image", "","Images (*.png *.xpm *.jpg *.bmp);;All Files (*)", options=options)
        if file_name:
            # Load the image
            self.image = cv2.imread(file_name)
            # Convert the image to RGB
            self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
    
    def detect(self):
        # Clear the image_label widget
        self.image_label.clear()
        # Perform Canny edge detection
        self.edges = cv2.Canny(self.image, self.param1_slider.value(), self.param1_slider.value() * 3)
        
        self.image_with_circles = self.image.copy()  # Create a copy of the image to draw the circles on
        
        # Use Hough transform to detect circles
        circles = cv2.HoughCircles(self.edges, cv2.HOUGH_GRADIENT, 1, 50, param1=self.param1_slider.value(), param2=self.param2_slider.value(), minRadius=0, maxRadius=0)
        
        # Check if circles were detected
        if circles is not None:
            # Convert the circle parameters from floating point to integer
            circles = np.round(circles[0, :]).astype("int")
            # Loop over the circles
            for (x, y, r) in circles:
                # Draw the circle in the image
                cv2.circle(self.image_with_circles, (x, y), r, (0, 0, 255), 2)  # <--- Modify this line to draw blue circles
        
        # Convert the image to QImage
        image = QImage(self.image_with_circles.data, self.image_with_circles.shape[1], self.image_with_circles.shape[0], QImage.Format_RGB888)
        # Set the image to the image_label widget
        self.image_label.setPixmap(QPixmap.fromImage(image))
        
        # Convert the edge map to QImage
        edges_image = QImage(self.edges.data, self.edges.shape[1], self.edges.shape[0], QImage.Format_Grayscale8)
        # Set the edge map to the edge_label widget
        self.edge_label.setPixmap(QPixmap.fromImage(edges_image))


In [2]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = DetectCircles()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\User\anaconda3\envs\machineVision\lib\site-packages\IPython\core\interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
